<a href="https://colab.research.google.com/github/PershinIlya/eeg-fm-eeg2025/blob/main/notebooks/labram_challenge1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>